<a href="https://colab.research.google.com/github/marketpsych/marketpsych/blob/main/notebooks/load_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Loading CSV files directly from SFTP

This notebook shows how to load MarketPsych's data with your SFTP credentials directly into a Jupyter Notebook. Note, however, that this notebook is an **alpha version**. For more robust testing, we recommend downloading the trialing flat files, as instructed by the MRNSupport, and then loading them into your own environment. 

---
## Settings
In order to have this example working, you'll need to install MarketPsych's library. Additionally, this notebook uses some widgets to facilitate with your navigation. To install the libraries and enable widgets, please run the following cell. 

In [ ]:
import sys
# Installs marketpsych library into your environment
!{sys.executable} -m pip install marketpsych --upgrade --quiet
# Installs ipywidget library into your environment
!{sys.executable} -m pip install ipywidgets --upgrade --quiet

## Libraries
from marketpsych import sftp
from marketpsych import mpwidgets

# Allows using the widgets
!{sys.executable} -m jupyter nbextension enable --py widgetsnbextension

from IPython.core.magic import register_cell_magic
from IPython.display import HTML, display

%load_ext autoreload
%autoreload 2

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


---
## Selecting your login credentials

Please input your credentials, i.e., the path to the `Key file`, and `UserID` as provided by the MRNSupport. 

<font color='blue'>**IMPORTANT**</font>       
<font color='blue'>1. Run the following cell. After running it, you should see two widgets.</font>  
<font color='blue'>2. Click on the uploader named `Key File`.</font>        
<font color='blue'>3. Check if your `User ID` is correct, if not, change it manually.</font>  
<font color='blue'>4. Move on to the next cell.</font>  

In [ ]:
cwdgts = mpwidgets.LoginWidgets()
cwdgts.display()

**WARNING**                         
When you run the following cell after waiting for too long, you may be given the following error:
```python
"ValueError": I/O operation on closed file.
```   

If so, please re-run the previous cell and upload your key. Otherwise, continue.

In [ ]:
# Creates client
client = sftp.connect(user=cwdgts.id_widget.value, key=cwdgts.key_widget.content)

---
## Loading the data

Finally, you can download the files directly into a pandas dataframe. The options can be defined through the 5 widgets below.

 - Select the checkbox if you are trialing the data. In some special cases, even if you are trialing, you may need to uncheck it (you can try both options in case of Permission errors).

 - The options for **Asset class** are:  

|Asset class   | Description|
|:-------------|:------------|
|`CMPNY`       | Individual companies|
|`CMPNY_AMER`  | Individual companies domiciled in America|
|`CMPNY_APAC`  | Individual companies domiciled in APAC|
|`CMPNY_EMEA`  | Individual companies domiciled in EMEA|
|`CMPNY_ESG`   | Individual companies (ESG package)|
|`CMPNY_GRP`   | Company groups and ETFs|
|`COM_AGR`     | Agricultural commodities|  
|`COM_ENM`     | Energy and Metals|
|`COU`         | Countries|
|`COU_ESG`     | Countries (ESG package)|
|`COU_MKT`     | Stock indices, sovereign bonds, real estate|
|`CRYPTO`      | Cryptocurrencies|
|`CUR`         | Currencies|  

 - The options for **frequencies** are:  
  
|Frequency  | Description| Use case |
|:----------|:-----------|:---------|
|`W365_UDAI`| Yearly lookback window and daily updates| ESG Core only |
|`WDAI_UDAI`| Daily lookback window and daily updates| Daily data stamped 30 minutes before the NYSE close|
|`WDAI_UHOU`| Daily lookback window and hourly updates| Daily data stamped hourly (in case you want daily data adjusted to your time-zone) |
|`W01M_U01M`| Minutely lookback window and minutely updates| Low-latency data (**WARNING:** extremely large datasets)|
 
- Regarding **dates**, simply select the start and end dates of interest. Note that for dates older than 2 months (with respect to your current date), the files are packaged in monthly batches. Hence, only the selection of month will matter and not the specific day. For example, if you select `2020-12-25` as the start or end date, the full `2020-12` month will be loaded anyways.

**WARNINGS**   
Loading large files such as CMPNY data with a long window-frame can take quite a while and take over your memory. Start by loading very short periods (e.g., one month of data), then moving to ever longer periods. 

Check your asset class permissions. If you try downloading data for which no access was provided, it will give a Permission error:
```python
"PermissionError": [Errno 13] Access denied
```   

<font color='blue'>Run the following cell ONCE. After it you'll be able to select the parameters. Once you have selected them, run the next cell.</font> 

In [ ]:
lwdgts = mpwidgets.LoaderWidgets()
lwdgts.display()

Checkbox(value=True, description='Trial')

Output()

In [ ]:
%%time

df = client.download(
    asset_class=sftp.AssetClass[lwdgts.asset_class_widget.value],
    frequency=sftp.Frequency[lwdgts.frequency_widget.value],
    start=lwdgts.start_date_widget.value,
    end=lwdgts.end_date_widget.value,
    trial=lwdgts.trial_check_widget.value,
    assets=tuple(lwdgts.assets_widget.value.split()),
    sources=tuple(lwdgts.data_type_widget.value)
)

display(df)

,id,assetCode,windowTimestamp,dataType,systemVersion,mentions,buzz,sentiment,negative,positive,...,overvaluedVsUndervalued,volatility,consumptionVolume,productionVolume,regulatoryIssues,supplyVsDemand,supplyVsDemandForecast,newExploration,safetyAccident,futureVsPast
0,mp:2020-12-01_20.30.00.News.COM_ENM.CRU,CRU,2020-12-01 20:30:00+00:00,News,MP:4.0.0,5585,26119.65,-0.018722,0.242021,0.223299,...,0.000077,0.023574,0.000345,-0.015506,0.000077,-0.011811,-0.012270,0.001091,0.000115,-0.249569
1,mp:2020-12-01_20.30.00.News_Social.COM_ENM.CRU,CRU,2020-12-01 20:30:00+00:00,News_Social,MP:4.0.0,8009,28548.95,-0.015762,0.239921,0.224159,...,-0.000105,0.023109,0.000280,-0.014361,0.000070,-0.010368,-0.011542,0.001314,0.000140,-0.236252
2,mp:2020-12-01_20.30.00.News_Headline.COM_ENM.CRU,CRU,2020-12-01 20:30:00+00:00,News_Headline,MP:4.0.0,289,468.00,-0.141026,0.305556,0.164530,...,NaN,0.021368,NaN,-0.056624,NaN,-0.048077,-0.012821,NaN,NaN,-0.103632
3,mp:2020-12-01_20.30.00.Social.COM_ENM.CRU,CRU,2020-12-01 20:30:00+00:00,Social,MP:4.0.0,2424,2429.30,0.016054,0.217347,0.233401,...,-0.002058,0.018112,-0.000412,-0.002058,NaN,0.005146,-0.003705,0.003705,0.000412,-0.093072
4,mp:2020-12-02_20.30.00.News.COM_ENM.CRU,CRU,2020-12-02 20:30:00+00:00,News,MP:4.0.0,4853,18488.80,-0.010574,0.237793,0.227219,...,0.000000,0.018930,0.001136,-0.015117,NaN,0.001244,0.002380,0.001298,0.000054,-0.265031
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,mp:2020-12-29_20.30.00.Social.COM_ENM.CRU,CRU,2020-12-29 20:30:00+00:00,Social,MP:4.0.0,1759,1798.50,0.063664,0.193217,0.256881,...,-0.002224,0.021129,0.001668,0.000000,NaN,-0.022519,-0.008896,0.003336,NaN,-0.071726
116,mp:2020-12-30_20.30.00.News.COM_ENM.CRU,CRU,2020-12-30 20:30:00+00:00,News,MP:4.0.0,4597,17358.55,0.059481,0.219200,0.278681,...,-0.000634,0.023231,0.001383,-0.003687,NaN,-0.021603,0.002132,0.001267,0.002708,-0.256401
117,mp:2020-12-30_20.30.00.News_Social.COM_ENM.CRU,CRU,2020-12-30 20:30:00+00:00,News_Social,MP:4.0.0,7281,19424.50,0.055651,0.217457,0.273109,...,-0.000978,0.022935,0.001313,-0.003269,NaN,-0.019743,0.001931,0.001493,0.002420,-0.240907
118,mp:2020-12-30_20.30.00.News_Headline.COM_ENM.CRU,CRU,2020-12-30 20:30:00+00:00,News_Headline,MP:4.0.0,295,745.50,0.039571,0.216633,0.256204,...,NaN,0.008719,NaN,-0.026157,NaN,-0.119383,0.000000,0.001341,0.001341,-0.225352


CPU times: user 101 ms, sys: 15.7 ms, total: 116 ms
Wall time: 4.59 s


If you can see a dataframe above, congratulations! You have downloaded some data into your notebook. From here, you can have fun exploring it. Below, you'll find a plotting tool for some simple understanding of the data.

---
## Visualizing the data

Below you can use the widgets to do some very basic exploration. A description of the widgets is given below.

- The **Data Type** field represents the type of content source(s) on which the RMAs are based. There are four possible values:
    - `News` for news sources (headlines and corpus)
    - `News_Headline` for the headline only of news sources
    - `Social` for social media sources
    - `News_Social` for the combined content               


- The **Anaytics** field represents the RMA. The actual values will depend on the asset class. Several types of indicators are provided:
    - Emotional indicators such as Anger, Fear and Joy
    - 'Economic' metrics including Earnings Forecast, Interest Rate Forecast, Long vs. Short 
    - ESG measures including CarbonEmissionsControversy, ManagementTrust, and WorkplaceSafety
    - etc. 


- The **Asset** field represents the asset of choice. To see all options, clear the cell. For a description of each asset, please search for the asset code in the User Guide or Eikon app.


- The **Roll. window** field represents the length in the smoothing function (a simple moving average).

The indicators are updated every minute for companies, sectors, regions, countries, commodities, indices, bonds, currencies and cryptocurrencies. They can be translated directly into spreadsheets or charts that can be monitored by traders, risk managers or analysts – or they can be plugged straight into your algorithms for low frequency or longer-term asset allocation or sector rotation decisions.

**WARNING**                
If your plot is empty, it is likely that there is no data for that combination of the three top variables. 

<font color='blue'>Select options for plotting the data after running the cell:</font> 

In [ ]:
swdgts = mpwidgets.SlicerWidgets(df)
swdgts.display()